# Input: View Name

In [39]:
# This is our input view.  
# It needs to be 2-part named using square bracket identifier delimiters
# e.g. "[export].[Board_Intake]""
qualified_view_name = "[export].[Board_Intake]"

# Imports

In [40]:
import pyodbc
import pandas as pd
import re

# Includes

In [41]:
%run common_functions

# Python Functions

In [42]:
def parse_object_name(qualified_view_name: str):
    """
    Parse a quoted/qualified object name of the form [SchemaName].[ViewName]
    and return (SchemaName, ViewName) without the brackets.
    """
    # Use regex to capture names inside square brackets
    matches = re.findall(r"\[([^\]]+)\]", qualified_view_name)
    
    if len(matches) == 2:
        return matches[0], matches[1]
    else:
        raise ValueError(f"Invalid qualified name format: {qualified_view_name}")

In [43]:
def run_batch_return_last_df(conn, sql: str, params=None):
    """Execute a multi-statement batch and return ONLY the last SELECT as a pandas DataFrame."""
    import pandas as pd
    with conn.cursor() as cur:
        cur.execute(sql, params or [])
        last_rows, last_cols = None, None
        while True:
            if cur.description is not None:          # this statement produced a result set
                last_cols  = [d[0] for d in cur.description]
                last_rows  = cur.fetchall()          # keep overwriting -> we keep the LAST
            if not cur.nextset():                    # advance to next statement's result set; stop at end
                break
    if last_rows is None:
        return pd.DataFrame()
    return pd.DataFrame.from_records(last_rows, columns=last_cols)


In [44]:
# --- helper to extract 'BULK' / 'DATA_SOURCE' / 'FORMAT' string values (handles ' or ")
def _find_arg(block: str, keyword: str):
    # Matches: KEYWORD [=] 'value'  or  "value"  (optional N prefix)
    pat = rf"\b{keyword}\b\s*(?:=\s*)?(?:N)?(?P<q>['\"])(?P<val>.*?)(?P=q)"
    m = re.search(pat, block, flags=re.IGNORECASE | re.DOTALL)
    return m.group("val").strip() if m else None

def parse_openrowset(block: str):
    return {
        "bulk":        _find_arg(block, "BULK"),
        "data_source": _find_arg(block, "DATA_SOURCE"),
        "format":      (_find_arg(block, "FORMAT") or None)
    }

# SQL Queries

## Our View Dependency SQL Query

In [45]:
# our view dependency query:
# We use view_name from above to filter the results.
sql_view_dependencies = """
;WITH cte AS
(
	--Root level ****************************************************
	SELECT 
		NestLevel = 0
		,QUOTENAME(SCHEMA_NAME(o.schema_id)) + N'.' + QUOTENAME(o.name) AS RootObjectName 
		,CAST(COALESCE(QUOTENAME(SCHEMA_NAME(o.schema_id)) + N'.' + QUOTENAME(o.name) + N'->' +
			-- optional server
			COALESCE(QUOTENAME(sed.referenced_server_name) + N'.', '') +
			-- optional DB
			COALESCE(QUOTENAME(sed.referenced_database_name) + N'.', '') +
			-- schema (not always there
			COALESCE(QUOTENAME(sed.referenced_schema_name) + N'.', '') +
			-- object
			QUOTENAME(sed.referenced_entity_name), QUOTENAME(SCHEMA_NAME(o.schema_id)) + N'.' + QUOTENAME(o.name)) AS NVARCHAR(MAX))
			AS CallHierarchy
		,SCHEMA_NAME(o.schema_id) AS ParentSchemaName
		,o.name AS ParentObjectName
		,o.type_desc AS ParentObjectType
		,sed.referenced_server_name AS ChildServerName
		,sed.referenced_database_name AS ChildDatabaseName
		,sed.referenced_id AS ChildObjectID
		,sed.referenced_schema_name AS ChildSchemaName
		,sed.referenced_entity_name AS ChildObjectName
		,oReferenced.type_desc AS ChildObjectType
	FROM
		sys.objects AS o
		LEFT JOIN sys.sql_expression_dependencies AS sed 
			ON(o.object_id = sed.referencing_id) 
		-- what is the referenced entity type?
		LEFT JOIN sys.objects AS oReferenced
			ON(sed.referenced_id = oReferenced.object_id)
	WHERE
		o.type IN ('FN', 'IF', 'TF', 'P', 'V')
		AND o.is_ms_shipped = 0

	-- OK - now we make it recursive - traverse the hierarchy
	UNION ALL

	-- decendant query ********************************
	SELECT 
		NestLevel = c.NestLevel + 1
		,c.RootObjectName
		,c.CallHierarchy + CAST(N'->' +
			-- optional server
			COALESCE(QUOTENAME(sed.referenced_server_name) + N'.', '') +
			-- optional DB
			COALESCE(QUOTENAME(sed.referenced_database_name) + N'.', '') +
			-- schema (not always there
			COALESCE(QUOTENAME(sed.referenced_schema_name) + N'.', '') +
			-- object
			QUOTENAME(sed.referenced_entity_name) AS NVARCHAR(MAX))
			AS CallHierarchy
		,c.ChildSchemaName AS ParentSchemaName
		,c.ChildObjectName AS ParentObjectName
		,c.ChildObjectType AS ParentObjectType
		,sed.referenced_server_name AS ChildServerName
		,sed.referenced_database_name AS ChildDatabaseName
		,sed.referenced_id AS ChildObjectID
		,sed.referenced_schema_name AS ChildSchemaName
		,sed.referenced_entity_name AS ChildObjectName
		--,sed.type_desc AS ChildObjectType
		,oReferenced.type_desc AS ChildObjectType
	FROM
		cte AS c 
		-- Take the previous iterations CTE Child/referenced and see if we find a parent/referencing 
		-- sql expr for it.
		-- This is the essence of the recursion.
		INNER JOIN sys.sql_expression_dependencies AS sed
			ON(c.ChildObjectID = sed.referencing_id)
		OUTER APPLY (SELECT * FROM sys.objects AS oReferenced1 WHERE sed.referenced_id = oReferenced1.object_id) AS oReferenced
	WHERE
		1=1
		AND (sed.referencing_id <> sed.referenced_id OR sed.referenced_id IS NULL) -- We DO need this to make sure it's not a recurive call, but also need to handle cross DB null referenced_id 
)
SELECT 
	@@SERVERNAME AS ServerName
	,DB_NAME() AS DatabaseName
	,c.*
INTO #dep
--	DependencyCollector.dbo.SQLDependency
FROM
	cte AS c
ORDER BY
	c.RootObjectName
	,c.NestLevel

-- ***************************************************************************************************************
-- Now we can select from #dep and query it however we want to:
SELECT
	*
FROM
	#dep AS d
WHERE
	d.RootObjectName = ? --'[export].[Board_Intake]'
ORDER BY
	d.RootObjectName
	,d.NestLevel
	,d.ChildSchemaName
	,d.ChildObjectName
"""

#print(sql_view_dependencies)

## Get object_id from schema and view name

In [46]:
# Query to get object_id from view name and schema name
sql_get_view_object_id = """
select 
	v.object_id
from
	sys.views as v
where
	v.name = ?
	and SCHEMA_NAME(v.schema_id) = ?
"""

## Get view definition for object_id

In [47]:
# Query to lookup view definitions for a view via object_id
# so we can parse out the OPENROWSETs that we find
sql_view_def = """
select
	sm.definition
from
	sys.sql_modules as sm
where
	sm.object_id = ? --(1711345161)
"""

# Get our Serverless SQL Connection Info, Including Creds

In [48]:
# Set our SQL Endpoint URL (prod), DB, user
serverless_server = "azwwwprodprdadapsyn01-ondemand.sql.azuresynapse.net"
database         = "sql"
username         = "util"

# Get our util SQL login/user password from AKV
secret_name = "util-sql-login-password"    

# Retrieve the secret value
password = mssparkutils.credentials.getSecret(kv_name, secret_name)

# (Optional) sanity check: do not print secrets!
print(f"Retrieved password for secret '{secret_name}' (length={len(password)})")

# Configure and build our connection (conn) object
conn_str = (
    "Driver={ODBC Driver 18 for SQL Server};"
    f"Server=tcp:{serverless_server},1433;"
    f"Database={database};"
    f"Uid={username};"
    f"Pwd={password};"
    "Encrypt=yes;TrustServerCertificate=no;Connection Timeout=60;"
)

conn = pyodbc.connect(conn_str)
conn.autocommit = True  # serverless-friendly; avoids implicit transactions
print("PyODBC Connection object conn built.")

# Get the dependecies for our input view

In [49]:
# query our dependencies and get them as a list of dicts.
# view_name is set at the top of the notebook.
pdf = run_batch_return_last_df(conn, sql_view_dependencies, qualified_view_name)
#pdf.head()
print(f"pdf.count() = {len(pdf)}")

# Get distinct rows
df_unique_children = (
    pdf[["ChildObjectID", "ChildSchemaName", "ChildObjectName"]]
    .drop_duplicates()
)

# rename columns
# Rename columns before converting
df_unique_children = df_unique_children.rename(
    columns={
        "ChildObjectID": "object_id",
        "ChildSchemaName": "schema_name",
        "ChildObjectName": "view_name"
    }
)

# Convert to list of dicts
view_list = df_unique_children.to_dict(orient="records")

# Find OPENROWSET refs in all of our views, including the root one

## Get the full list of views to analyze, incuding the root one (input)

In [50]:
# Parse out the schema name and view name from the quoted qualified name
schema_name, view_name = parse_object_name(qualified_view_name)
print(f"view_name = {view_name}")
print(f"schema_name = {schema_name}")

# Get the object_id for our root view
pdf = run_batch_return_last_df(conn, sql_get_view_object_id, [view_name, schema_name])
#print(f"pdf.count() = {len(pdf)}")
#display(pdf)
root_object_id = pdf["object_id"].iloc[0]
print(f"root_object_id = {root_object_id}")

# Add this root view info to the list of object_ids for child views 
view_list.append({
    "object_id": root_object_id,
    "schema_name": schema_name,
    "view_name": view_name
})

print("We will be looking at these views:")
print(view_list)

## Get the view definition of each and extract the OPENROWSET call

In [51]:
print("Parsing view DDL for OPENROWSET calls...")
parsed_rows = []
for view in view_list:
    object_id = int(view["object_id"])
    schema_name = view["schema_name"]
    view_name = view["view_name"]

    print(f"Processing object_id {object_id}: [{schema_name}].[{view_name}]")

    # Get the view DDL (definition) for this view
    pdf_view_def = run_batch_return_last_df(conn, sql_view_def, [object_id])
    view_definition = pdf_view_def["definition"].iloc[0]

    # TODO Now get all of the OPENROWSET calls from the defintion 
    # Find each OPENROWSET(...) call (no nested parentheses inside the arg list)
    openrowset_pattern = re.compile(r"OPENROWSET\s*\([^)]*\)", re.IGNORECASE | re.DOTALL)

    calls = openrowset_pattern.findall(view_definition)
    # calls is a list of strings like: "OPENROWSET( BULK '...', DATA_SOURCE = '...', FORMAT = 'DELTA' )"   

    # Extract the BULK, DATA_SOURCE and FORMAT parameters for each OPENROWSET.
    if len(calls) == 0:
        print(f"No direct OPENROWSET calls found in object_id {object_id}: [{schema_name}].[{view_name}].")
        continue

    for call in calls:
        info = parse_openrowset(call)
        # optional normalization
        if info["format"]:
            info["format"] = info["format"].upper()
        parsed_rows.append({
            "object_id": object_id,
            "schema_name": schema_name,
            "view_name": view_name,
            **info
        })

print("Finished parsing view DDL for OPENROWSET calls")

## Display our findings - we can export this to CSV, etc.

In [52]:
# Print our findings
#print(parsed_rows)

# Use your list variable here:
rows = parsed_rows  # or: rows = all_rows

cols = ["object_id", "schema_name", "view_name", "bulk", "data_source", "format"]
df = pd.DataFrame(rows)[cols].sort_values(
    ["schema_name", "view_name", "bulk", "data_source"]
).reset_index(drop=True)

# Notebook-friendly display
display(df)              